This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to overview](end-conditions.ipynb)

# Not-A-Knot End Conditions

We are looking only at the non-uniform case here,
it's easy to get to the uniform case by setting $\Delta_i = 1$.

In [ ]:
import sympy as sp
sp.init_printing(order='rev-lex')

[utility.py](utility.py)

In [ ]:
from utility import NamedExpression

In [ ]:
t = sp.symbols('t')

## Begin

first polynomial segment: $\boldsymbol{p}_0(t)$, $t_0 \le t \le t_1$

second polynomial segment: $\boldsymbol{p}_1(t)$, $t_1 \le t \le t_2$

In [ ]:
t0, t1, t2 = sp.symbols('t:3')

In [ ]:
#a0, a1, b0, b1, c0, c1, d0, d1 = sp.symbols('a:dbm0:2')

In [ ]:
b_monomial = sp.Matrix([t**3, t**2, t, 1]).T

In [ ]:
p0 = NamedExpression(
    'pbm0',
    b_monomial.dot(sp.symbols('a:dbm0')[::-1]).subs(t, (t - t0) / (t1 - t0)))
p1 = NamedExpression(
    'pbm1',
    b_monomial.dot(sp.symbols('a:dbm1')[::-1]).subs(t, (t - t1) / (t2 - t1)))
pd0 = p0.diff(t)
pd1 = p1.diff(t)
display(p0, p1, pd0, pd1)

???

\begin{align}
\boldsymbol{x}_0 &= \boldsymbol{p}_0(t_0)\\
\boldsymbol{x}_1 &= \boldsymbol{p}_0(t_1)\\
\boldsymbol{\dot{x}}_0 &= \boldsymbol{p}_0'(t_0)\\
\boldsymbol{\dot{x}}_1 &= \boldsymbol{p}_0'(t_1)
\end{align}

In [ ]:
pdd0 = pd0.diff(t)
pdd1 = pd1.diff(t)
display(pdd0, pdd1)

In [ ]:
equations_begin = [
    p0.evaluated_at(t, t0).with_name('xbm0'),
    p0.evaluated_at(t, t1).with_name('xbm1'),
    pd0.evaluated_at(t, t0).with_name('xbmdot0'),
    pd0.evaluated_at(t, t1).with_name('xbmdot1'),
    p1.evaluated_at(t, t1).with_name('xbm1'),
    p1.evaluated_at(t, t2).with_name('xbm2'),
    pd1.evaluated_at(t, t1).with_name('xbmdot1'),
    #pd1.evaluated_at(t, t2).with_name('xbmdot2'),
    sp.Eq(pdd0.evaluated_at(t, t1).expr, pdd1.evaluated_at(t, t1).expr),
]

only for display purposes,
the calculations are still done with $t_i$

In [ ]:
deltas_begin = {
    t1 - t0: sp.Symbol('Delta0'),
    t2 - t1: sp.Symbol('Delta1'),
    #2 * (t1 - t0): 2 * sp.Symbol('Delta0'),
    #2 * (t2 - t1): 2 * sp.Symbol('Delta1'),
}

In [ ]:
for e in equations_begin:
    display(e.subs(deltas_begin))

In [ ]:
coefficients_begin = sp.solve(equations_begin, sp.symbols('a:dbm0:2'))

In [ ]:
for c, e in coefficients_begin.items():
    display(NamedExpression(c, e.subs(deltas_begin)))

In [ ]:
delta0, delta1 = sp.symbols('Delta:2')

In [ ]:
deltas_begin = {
    t1 - t0: delta0,
    t2 - t1: delta1,
    sp.expand((t1 - t0)**2): delta0**2,
    sp.expand((t1 - t0)**3): delta0**3,
    sp.expand((t2 - t1)**3 * (t1 - t0)**2): delta1**3 * delta0**2,
    
    sp.expand((t1 - t0) * delta1**3): delta0 * delta1**3,
}

In [ ]:
pddd0 = pdd0.diff(t)
pddd1 = pdd1.diff(t)
display(pddd0, pddd1)

In [ ]:
expr = pddd0.evaluated_at(t, t1).expr - pddd1.evaluated_at(t, t1).expr
expr

In [ ]:
expr = expr.subs(coefficients_begin).subs(deltas_begin).simplify().factor()
expr

In [ ]:
expr *= sp.denom(expr)
expr

In [ ]:
expr /= 6

In [ ]:
expr

In [ ]:
expr.collect(sp.symbols('xbmdot:2'))

## End

$N$ vertices, $N-1$ polynomial segments

last polynomial: $\boldsymbol{p}_{N-2}(t)$, $t \in [t_{N-2}, t_{N-1}]$

To simplify the notation a bit,
let's assume we have $N = 10$ vertices,
which makes $\boldsymbol{p}_8$ the last polynomial segment.

Luckily, that's symmetric to the result we got above.